In [1]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
import torch
import scipy.io
import numpy as np
# import torch.nn.functional as F  # For Softmax of outputs in testing, not used now

In [4]:
deap_de_path = '../../methods/DE_CNN/1D_dataset/'

### Check Data/Labels and shapes for subject 1

In [5]:
s01 = scipy.io.loadmat(deap_de_path + 'DE_s01.mat')
for i, key in enumerate(s01):
    print(key)

__header__
__version__
__globals__
base_data
data
valence_labels
arousal_labels


In [6]:
X_s01 = s01['data']
y_s01_valence = s01['valence_labels']
y_s01_arousal = s01['arousal_labels']

In [7]:
print(X_s01.shape)
print(y_s01_valence.shape)
print(np.transpose(y_s01_valence).shape)

(2400, 4, 32)
(1, 2400)
(2400, 1)


In [8]:
np.unique(np.transpose(y_s01_valence), return_counts=True)

(array([0., 1.]), array([1260, 1140], dtype=int64))

In [9]:
print(1260/2400*100, '% valence=0')
print(1140/2400*100, '% valence=1')

52.5 % valence=0
47.5 % valence=1


## Define DEAP Dataset

In [10]:
from torch.utils.data import Dataset

In [11]:
class DEAPDataset(Dataset):
    def __init__(self, data, labels):
        self.X = data
        self.y = labels
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## DataLoader

In [12]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [13]:
def train_test_iterators(deap_dataset, batch_size=32, train_split=0.8, test_split=0.2):
    train_n_elems = int(train_split * len(deap_dataset))
    test_n_elems = int(test_split * len(deap_dataset))
    
    deap_train, deap_test = random_split(deap_dataset, [train_n_elems, test_n_elems])
    
    train_dataloader = DataLoader(deap_train, batch_size=batch_size, shuffle=True, num_workers=0)
    test_dataloader = DataLoader(deap_test, batch_size=batch_size, shuffle=True, num_workers=0)
    
    return train_dataloader, test_dataloader

## Train Loop

In [14]:
def train_model(model, criterion, optimizer, train_dataloader, model_path, writer, subj,
                num_epochs=50, skip_training=False, min_loss=1000,  acc_n_steps=20):
    running_loss = 0.0
    running_correct = 0
    n_total_steps = len(train_dataloader)
    
    if not skip_training:
        for epoch in range(num_epochs):
            model.train()
            for i, (data, labels) in enumerate(train_dataloader):
                data = data.to(device).float().permute(0, 2, 1)
                labels = labels.to(device).float()

                outputs = model(data)   
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step() 
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                running_correct += (predicted == labels.reshape(-1,)).sum().item()

                if (i+1) % acc_n_steps == 0:
                        writer.add_scalar('training loss', running_loss / acc_n_steps, epoch * n_total_steps + i)
                        running_accuracy = running_correct / acc_n_steps / predicted.size(0) * 100
                        writer.add_scalar('training accuracy', running_accuracy, epoch * n_total_steps + i)                        
                        running_correct = 0
                        running_loss = 0.0
                        print(f's{subj:02}, Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.6f}, Accuracy: {running_accuracy:.2f}')

                if epoch > 5 and loss.item() < min_loss:
                        min_loss = loss.item()
                        torch.save({
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'min_loss': min_loss
                        }, model_path)
                        print(f'Saved checkpoint - loss: {min_loss:.12f}')          
        print('Finished Training')

## Test Loop

In [15]:
def test_model(model, test_dataloader, load_model=False):
    with torch.no_grad():
        model.eval()  
        if load_model:
            checkpoint = torch.load(model_path)
            model.load_state_dict(checkpoint['model_state_dict'])

        n_correct = 0
        n_samples = 0

        for data, labels in test_dataloader:
            data = data.to(device).float().permute(0, 2, 1)
            labels = labels.to(device).float()

            outputs = model(data)

            # torch.max returns (value, index)
            _, predicted = torch.max(outputs, 1)

            n_samples += labels.size(0)
            n_correct += (predicted == labels.reshape(-1,)).sum().item()              

    acc = n_correct / n_samples * 100.0 
    print(f'Testing accuracy of the network: {acc:.2f}%')
    
    return acc

## Subject Dependent Loop (1-32 subjects)

## Define BiHDM Model

In [16]:
from Models_DEAP import BiHDM

In [17]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
device

device(type='cpu')

### BiHDM Initialization Parameters

In [18]:
num_epochs = 50

In [19]:
batch_size=32
train_split=0.8
test_split=0.2

In [20]:
chosen_labels = 'valence'  # valence/arousal/all
chosen_n_classes = 1

In [21]:
hidden_size = 32
num_layers = 2
input_size = 4
n_classes = chosen_n_classes

# batch_first=False
# bidirectional=False

fc_input=448
fc_hidden=96

### Loss Function

In [22]:
criterion = torch.nn.BCEWithLogitsLoss()

### Adam Optimizer Parameters

In [23]:
lr=0.001
betas=(0.9, 0.999)

### Train/Test loop modes

In [24]:
skip_training = False
resume_training = False

In [25]:
load_model = True  # During the test loop the best model (which was saved during the training) will be loaded

## Training BiHDM

In [26]:
n_subjects = 32

In [27]:
overall_accuracy_list = np.empty((n_subjects,))

In [28]:
for subj in range(1, n_subjects+1):  # Subjects 1-32 in DEAP
    print(f'***Processing Subject {subj}/{n_subjects}***')
    
    # writer = SummaryWriter(f'C:/Users/Alex/git/EEG-emotion/methods/Saliency_Emotion_EEG/TensorBoard/deap_bihdm_subj-dep_s{subj:02}')   
    writer = SummaryWriter(f'C:/Users/Alex/git/EEG-emotion/methods/Saliency_Emotion_EEG/TensorBoard/deap_bihdm_subj-dep_s{subj:02}')   
    model_path = f'./saved_models/BiHDM-Model-Subj-Dep_s{subj:02}.pth'   
    
    subj_data = scipy.io.loadmat(deap_de_path + f'DE_s{subj:02}.mat')
    X = subj_data['data']
    
    if chosen_labels == 'valence':
        y = np.transpose(subj_data['valence_labels'])
    elif chosen_labels == 'arousal':
        y = np.transpose(subj_data['arousal_labels'])
        
    deap_dataset = DEAPDataset(X, y)
    train_dataloader, test_dataloader = train_test_iterators(deap_dataset, batch_size, train_split, test_split)
    n_total_steps = len(train_dataloader)  # (2400*0.8)/32=60  or (trials*t_windows*train_split)/batch_size
    acc_n_steps = n_total_steps / 2  # 60/2=30
    
    model = BiHDM(hidden_size=hidden_size, num_layers=num_layers, input_size=input_size, 
              fc_input=fc_input, fc_hidden=fc_hidden, n_classes=n_classes) 
    model.to(device).float()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)
    
    if not skip_training:
        if resume_training:
            #checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
            checkpoint = torch.load(model_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict']) 
            min_loss = checkpoint['min_loss']
        else:
            min_loss = 1000
            
        train_model(model, criterion, optimizer, train_dataloader, model_path, writer, subj,
                    num_epochs, skip_training, min_loss,  acc_n_steps)
    writer.close()
    
    subj_acc = test_model(model, test_dataloader, load_model)
    
    overall_accuracy_list[subj-1] = subj_acc
    
    %notify -m "Subject Completed!"  

***Processing Subject 1/32***
s01, Epoch [1/50], Step [30/60], Loss: 0.720637, Accuracy: 54.37
s01, Epoch [1/50], Step [60/60], Loss: 0.691473, Accuracy: 51.15
s01, Epoch [2/50], Step [30/60], Loss: 0.683412, Accuracy: 53.65
s01, Epoch [2/50], Step [60/60], Loss: 0.656550, Accuracy: 51.88
s01, Epoch [3/50], Step [30/60], Loss: 0.686478, Accuracy: 52.81
s01, Epoch [3/50], Step [60/60], Loss: 0.617198, Accuracy: 52.71
s01, Epoch [4/50], Step [30/60], Loss: 0.586562, Accuracy: 53.44
s01, Epoch [4/50], Step [60/60], Loss: 0.651287, Accuracy: 52.08
s01, Epoch [5/50], Step [30/60], Loss: 0.656830, Accuracy: 54.79
s01, Epoch [5/50], Step [60/60], Loss: 0.593794, Accuracy: 50.73
s01, Epoch [6/50], Step [30/60], Loss: 0.675639, Accuracy: 50.62
s01, Epoch [6/50], Step [60/60], Loss: 0.562411, Accuracy: 54.90
Saved checkpoint - loss: 0.642805218697
Saved checkpoint - loss: 0.582706749439
Saved checkpoint - loss: 0.526855230331
Saved checkpoint - loss: 0.441415458918
s01, Epoch [7/50], Step [30/60

s01, Epoch [49/50], Step [60/60], Loss: 0.004834, Accuracy: 51.98
s01, Epoch [50/50], Step [30/60], Loss: 0.005365, Accuracy: 52.92
Saved checkpoint - loss: 0.001900502597
s01, Epoch [50/50], Step [60/60], Loss: 0.004204, Accuracy: 52.60
Finished Training
Testing accuracy of the network: 51.46%


<IPython.core.display.Javascript object>

***Processing Subject 2/32***
s02, Epoch [1/50], Step [30/60], Loss: 0.682221, Accuracy: 46.04
s02, Epoch [1/50], Step [60/60], Loss: 0.584750, Accuracy: 42.08
s02, Epoch [2/50], Step [30/60], Loss: 0.618114, Accuracy: 43.75
s02, Epoch [2/50], Step [60/60], Loss: 0.716750, Accuracy: 44.38
s02, Epoch [3/50], Step [30/60], Loss: 0.474504, Accuracy: 44.48
s02, Epoch [3/50], Step [60/60], Loss: 0.771291, Accuracy: 43.65
s02, Epoch [4/50], Step [30/60], Loss: 0.748520, Accuracy: 45.10
s02, Epoch [4/50], Step [60/60], Loss: 0.512581, Accuracy: 43.02
s02, Epoch [5/50], Step [30/60], Loss: 0.668893, Accuracy: 43.96
s02, Epoch [5/50], Step [60/60], Loss: 0.597208, Accuracy: 44.17
s02, Epoch [6/50], Step [30/60], Loss: 0.619918, Accuracy: 44.17
s02, Epoch [6/50], Step [60/60], Loss: 0.478283, Accuracy: 43.96
Saved checkpoint - loss: 0.634822070599
Saved checkpoint - loss: 0.493715554476
Saved checkpoint - loss: 0.446155488491
s02, Epoch [7/50], Step [30/60], Loss: 0.612797, Accuracy: 44.58
s02, 

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

***Processing Subject 31/32***
s31, Epoch [1/50], Step [30/60], Loss: 0.678025, Accuracy: 42.08
s31, Epoch [1/50], Step [60/60], Loss: 0.665028, Accuracy: 43.54
s31, Epoch [2/50], Step [30/60], Loss: 0.698470, Accuracy: 44.48
s31, Epoch [2/50], Step [60/60], Loss: 0.669549, Accuracy: 41.15
s31, Epoch [3/50], Step [30/60], Loss: 0.663665, Accuracy: 44.17
s31, Epoch [3/50], Step [60/60], Loss: 0.626421, Accuracy: 41.46
s31, Epoch [4/50], Step [30/60], Loss: 0.685273, Accuracy: 42.50
s31, Epoch [4/50], Step [60/60], Loss: 0.686233, Accuracy: 43.12
s31, Epoch [5/50], Step [30/60], Loss: 0.664852, Accuracy: 43.44
s31, Epoch [5/50], Step [60/60], Loss: 0.558792, Accuracy: 42.19
s31, Epoch [6/50], Step [30/60], Loss: 0.631197, Accuracy: 42.29
s31, Epoch [6/50], Step [60/60], Loss: 0.596587, Accuracy: 43.33
Saved checkpoint - loss: 0.554466366768
Saved checkpoint - loss: 0.533375978470
Saved checkpoint - loss: 0.497487515211
s31, Epoch [7/50], Step [30/60], Loss: 0.665144, Accuracy: 41.56
s31,

<IPython.core.display.Javascript object>

***Processing Subject 32/32***
s32, Epoch [1/50], Step [30/60], Loss: 0.705071, Accuracy: 49.69
s32, Epoch [1/50], Step [60/60], Loss: 0.709426, Accuracy: 50.52
s32, Epoch [2/50], Step [30/60], Loss: 0.655757, Accuracy: 48.96
s32, Epoch [2/50], Step [60/60], Loss: 0.636097, Accuracy: 51.25
s32, Epoch [3/50], Step [30/60], Loss: 0.635419, Accuracy: 49.58
s32, Epoch [3/50], Step [60/60], Loss: 0.711676, Accuracy: 50.62
s32, Epoch [4/50], Step [30/60], Loss: 0.691184, Accuracy: 50.62
s32, Epoch [4/50], Step [60/60], Loss: 0.620903, Accuracy: 49.58
s32, Epoch [5/50], Step [30/60], Loss: 0.632692, Accuracy: 48.54
s32, Epoch [5/50], Step [60/60], Loss: 0.582041, Accuracy: 51.67
s32, Epoch [6/50], Step [30/60], Loss: 0.632489, Accuracy: 51.15
s32, Epoch [6/50], Step [60/60], Loss: 0.530885, Accuracy: 49.06
Saved checkpoint - loss: 0.579425275326
Saved checkpoint - loss: 0.544206202030
Saved checkpoint - loss: 0.523985266685
Saved checkpoint - loss: 0.521289169788
Saved checkpoint - loss: 0.51

<IPython.core.display.Javascript object>

In [29]:
overall_accuracy_list

array([51.45833333, 48.75      , 45.83333333, 60.41666667, 37.70833333,
       20.        , 30.41666667, 41.45833333, 52.5       , 52.91666667,
       37.5       , 48.33333333, 57.5       , 50.625     , 51.45833333,
       63.95833333, 41.66666667, 41.66666667, 46.45833333, 43.125     ,
       47.29166667, 57.08333333, 33.95833333, 60.        , 58.125     ,
       35.83333333, 23.54166667, 37.08333333, 43.54166667, 30.625     ,
       41.25      , 49.58333333])

In [30]:
for subj_idx, acc in enumerate(overall_accuracy_list):
    print(f's{subj_idx+1:02} - {acc:.2f}', sep='\t')

s01 - 51.46
s02 - 48.75
s03 - 45.83
s04 - 60.42
s05 - 37.71
s06 - 20.00
s07 - 30.42
s08 - 41.46
s09 - 52.50
s10 - 52.92
s11 - 37.50
s12 - 48.33
s13 - 57.50
s14 - 50.62
s15 - 51.46
s16 - 63.96
s17 - 41.67
s18 - 41.67
s19 - 46.46
s20 - 43.12
s21 - 47.29
s22 - 57.08
s23 - 33.96
s24 - 60.00
s25 - 58.13
s26 - 35.83
s27 - 23.54
s28 - 37.08
s29 - 43.54
s30 - 30.63
s31 - 41.25
s32 - 49.58


In [31]:
%%notify -m "Evaluation Completed!"

mean_overall_accuracy = np.mean(overall_accuracy_list)
print(mean_overall_accuracy)

45.05208333333333


<IPython.core.display.Javascript object>